In [5]:
import pandas as pd
import numpy as np

In [7]:
import os

### Importing and Labeling dataset

In [9]:
dataset = r'D:/Downloads/Emotion detection dataset'

In [11]:
def create_label(base_dir):
    data = []
    emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    for emotion in emotions:
        emotion_dir = os.path.join(base_dir, emotion)
        
        # Iterate through all images in the emotion folder
        for img in os.listdir(emotion_dir):
            if img.endswith('.jpg'):  # Add supported image extensions
                img_path = os.path.join(emotion_dir, img)
                data.append([img_path, emotion])

    # Create a DataFrame with two columns: 'image_path' and 'emotion'
    df = pd.DataFrame(data, columns=['image_path', 'emotion'])
    return df

In [15]:
train_set = create_label(os.path.join(dataset, 'train'))
test_set = create_label(os.path.join(dataset, 'test'))

In [19]:
train_set.to_csv('train_labels',index = False)

In [21]:
test_set.to_csv('test_labels',index = False)

In [23]:
train_set

,image_path,emotion
0,D:/Downloads/Emotion detection dataset\train\a...,angry
1,D:/Downloads/Emotion detection dataset\train\a...,angry
2,D:/Downloads/Emotion detection dataset\train\a...,angry
3,D:/Downloads/Emotion detection dataset\train\a...,angry
4,D:/Downloads/Emotion detection dataset\train\a...,angry
...,...,...
28704,D:/Downloads/Emotion detection dataset\train\s...,surprise
28705,D:/Downloads/Emotion detection dataset\train\s...,surprise
28706,D:/Downloads/Emotion detection dataset\train\s...,surprise
28707,D:/Downloads/Emotion detection dataset\train\s...,surprise


In [28]:
train_df = pd.read_csv('train_labels')
test_df = pd.read_csv('test_labels')

In [30]:
train_df

,image_path,emotion
0,D:/Downloads/Emotion detection dataset\train\a...,angry
1,D:/Downloads/Emotion detection dataset\train\a...,angry
2,D:/Downloads/Emotion detection dataset\train\a...,angry
3,D:/Downloads/Emotion detection dataset\train\a...,angry
4,D:/Downloads/Emotion detection dataset\train\a...,angry
...,...,...
28704,D:/Downloads/Emotion detection dataset\train\s...,surprise
28705,D:/Downloads/Emotion detection dataset\train\s...,surprise
28706,D:/Downloads/Emotion detection dataset\train\s...,surprise
28707,D:/Downloads/Emotion detection dataset\train\s...,surprise


In [32]:
test_df

,image_path,emotion
0,D:/Downloads/Emotion detection dataset\test\an...,angry
1,D:/Downloads/Emotion detection dataset\test\an...,angry
2,D:/Downloads/Emotion detection dataset\test\an...,angry
3,D:/Downloads/Emotion detection dataset\test\an...,angry
4,D:/Downloads/Emotion detection dataset\test\an...,angry
...,...,...
7173,D:/Downloads/Emotion detection dataset\test\su...,surprise
7174,D:/Downloads/Emotion detection dataset\test\su...,surprise
7175,D:/Downloads/Emotion detection dataset\test\su...,surprise
7176,D:/Downloads/Emotion detection dataset\test\su...,surprise


### Preprocessing the data

In [43]:
!pip install opencv-python


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 435.7 kB/s eta 0:01:30
   ---------------------------------------- 0.1/38.8 MB 751.6 kB/s eta 0:00:52
   ---------------------------------------- 0.2/38.8 MB 1.3 MB/s eta 0:00:30
   ---------------------------------------- 0.4/38.8 MB 1.8 MB/s eta 0:00:22
    --------------------------------------- 0.5/38.8 MB 2.0 MB/s eta 0:00:20
    --------------------------------------- 0.7/38.8 MB 2.4 MB/s eta 0:00:17
    --------------------------------------- 0.8/38.8 MB 2.5 MB/s eta 0:00:16
   - -------------------------------------- 1.1/38.8 MB 2.8 MB/s eta 0:00:14
   - -------------------------------------- 1.3/38.8 MB 2.9 MB/s eta 0:00:13
   - -------------------------------------- 1.5/38.8 MB 3.2 MB/s eta 0:00:12
   - -------------------------------------- 1.8/38.8 MB 3.4 MB/s eta 0:00:11
   -- ---

In [47]:
!pip install tensorflow

     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.2/385.2 MB 6.3 MB/s eta 0:01:02
   ---------------------------------------- 0.5/385.2 MB 5.6 MB/s eta 0:01:09
   ---------------------------------------- 0.7/385.2 MB 5.2 MB/s eta 0:01:14
   ---------------------------------------- 1.4/385.2 MB 7.3 MB/s eta 0:00:53
   ---------------------------------------- 2.3/385.2 MB 9.1 MB/s eta 0:00:42
   ---------------------------------------- 3.1/385.2 MB 11.1 MB/s eta 0:00:35
   ---------------------------------------- 4.0/385.2 MB 12.1 MB/s eta 0:00:32
    --------------------------------------- 4.9/385.2 MB 12.7 MB/s eta 0:00:31
    --------------------------------------- 5.9/385.2 MB 13.4 MB/s eta 0:00:29
    --------------------------------------- 6.6/385.2 MB 14.2 MB/s eta 

In [50]:
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Parameters
image_size = (48, 48)  
num_emo = 7  # Number of emotion categories

# Function to preprocess grayscale images
def preprocess_images(df):
    images = []
    labels = []
    
    for index, row in df.iterrows():
        # Load image (already in grayscale)
        img = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
        
        # Resize image to the target size
        img_resized = cv2.resize(img, image_size)
        
        # Normalize pixel values (0 to 255 -> 0 to 1)
        img_normalized = img_resized / 255.0
        
        # Append image and its corresponding label
        images.append(img_normalized)
        labels.append(row['emotion'])

    # Convert images and labels to numpy arrays
    images = np.array(images).reshape(-1, image_size[0], image_size[1], 1)  # 1 -> grayscale
    labels = np.array(labels)

    return images, labels

# Load train and test datasets
X_train, y_train = preprocess_images(train_df)
X_test, y_test = preprocess_images(test_df)

# One-hot encode the labels
y_train_encoded = to_categorical(pd.factorize(y_train)[0], num_classes=num_emo)
y_test_encoded = to_categorical(pd.factorize(y_test)[0], num_classes=num_emo)

# Split training data into train and test(validation) sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_encoded, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation(X_val) data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")


Training data shape: (22967, 48, 48, 1)
Validation data shape: (5742, 48, 48, 1)
Test data shape: (7178, 48, 48, 1)


## Model Training

#### 1. Convolutional Neural Networks (CNNs)
CNN Architecture: Traditional CNN architectures (like VGG16, ResNet, or Inception) can be fine-tuned for emotion detection tasks. These models are effective for feature extraction from images.
Transfer Learning: Pre-trained models on large datasets (like ImageNet) can be fine-tuned on your specific emotion dataset for better performance.
#### 2. Facial Landmark Detection + CNN
Combine facial landmark detection (e.g., using Dlib or OpenCV) to extract key points on the face, then use those landmarks as input features to a CNN. This can improve performance by focusing on relevant facial regions.
#### 3. Recurrent Neural Networks (RNNs)
LSTM/GRU Models: For Video Data , using RNNs with Long Short-Term Memory (LSTM) or Gated Recurrent Unit (GRU) can capture temporal dependencies in facial expressions.
#### 4. Hybrid Models
Combining CNNs with RNNs (e.g., CNN followed by LSTM) can leverage spatial and temporal features, particularly useful in video emotion detection.
#### 5. Ensemble Methods
Using ensemble techniques that combine multiple models can improve accuracy and robustness in emotion detection tasks.

## Evaluation Metrics
When evaluating the performance of your model, consider using metrics like:

1. Accuracy
2. Precision, Recall, and F1 Score
3. Confusion Matrix
4. AUC-ROC Curve